### Extract Bonnet & Blowhead coordinates

In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model, save_model
from sklearn.model_selection import train_test_split

In [3]:
test_image_names = pd.read_csv('dataset/sample_submission.csv')['Image']
test_image_names.values

array(['w_1947.jpg', 'w_11096.jpg', 'w_10973.jpg', ..., 'w_5278.jpg',
       'w_9218.jpg', 'w_9316.jpg'], dtype=object)

### Load Train/Test Data

In [4]:
def load_train_images_to_dataframe(folder_path):
    # Initialize an empty list to store image information
    image_data = []
    image_file_name = []

    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    counter = 0
    # Iterate over each file
    for file_name in files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)
        
        # file_name_without_resized = file_name[8:]
        
        # To filter out the test images
        # if file_name_without_resized not in test_image_names.values:
        if file_name not in test_image_names.values:
            counter += 1
            # Read the image
            img = cv2.imread(file_path)
            
            # # Check if the image is loaded properly
            if img is None:
                print(f"Error: Image {file_name} not loaded!")
                continue
            image_file_name.append(file_name)
            image_data.append(img)

    # Create a DataFrame from the list of image information
    df = np.array(image_data)
    
    return image_file_name, df

def load_test_images_to_dataframe(folder_path):
    # Initialize an empty list to store image information
    image_data = []
    image_file_name = []
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    counter = 0
    # Iterate over each file
    for file_name in files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)
        
        # file_name_without_resized = file_name[8:]
        
        # if file_name_without_resized in test_image_names.values:
        if file_name in test_image_names.values:
            counter += 1
            # Read the image
            img = cv2.imread(file_path)
            
            # Check if the image is loaded properly
            if img is None:
                print(f"Error: Image {file_name} not loaded!")
                continue
            
            image_data.append(img)
            image_file_name.append(file_name)
            
    # Create a DataFrame from the list of image information
    df = np.array(image_data)
    
    return image_file_name, df

In [5]:
# Load the first CSV file with scaling ratios
scaling_data = pd.read_csv('img_scale_ratio_info.csv')

# Load the second CSV file with image data
image_data = pd.read_csv('annotations/train_with_points.csv')

# Merge the two dataframes based on the 'Image' and 'file_name' columns
merged_data = pd.merge(image_data, scaling_data, left_on='Image', right_on='file_name')

# Apply scaling ratios to the coordinate columns
merged_data['bonnet_tip_x_rescaled'] = merged_data['bonnet_tip_x'] * merged_data['x_scale_ratio']
merged_data['bonnet_tip_y_rescaled'] = merged_data['bonnet_tip_y'] * merged_data['y_scale_ratio']
merged_data['blowhead_x_rescaled'] = merged_data['blowhead_x'] * merged_data['x_scale_ratio']
merged_data['blowhead_y_rescaled'] = merged_data['blowhead_y'] * merged_data['y_scale_ratio']

# Select and reorder the columns for the final CSV file
final_data = merged_data[['Image', 'whaleID', 'bonnet_tip_x_rescaled', 'bonnet_tip_y_rescaled', 'blowhead_x_rescaled', 'blowhead_y_rescaled']]

# Write the final data to a CSV file
final_data.to_csv('annotations/train_with_points_scaled.csv', index=False)

In [6]:
def load_data(csv_file):
    df = pd.read_csv(csv_file)
    whale_ids = df['whaleID']
    bonnet_tip_coords = df[['bonnet_tip_x_rescaled', 'bonnet_tip_y_rescaled']]
    blowhead_coords = df[['blowhead_x_rescaled', 'blowhead_y_rescaled']]
    return whale_ids, bonnet_tip_coords, blowhead_coords

In [7]:
image_folder = 'dataset/imgs256x256#2'
csv_file = 'annotations/train_with_points_scaled.csv'

test_image_names = pd.read_csv('dataset/sample_submission.csv')['Image']
img_file_name, images = load_train_images_to_dataframe(image_folder)
whale_ids, bonnet_tip_coords, blowhead_coords = load_data(csv_file)

In [8]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, pd.concat([bonnet_tip_coords, blowhead_coords], axis=1), test_size=0.1, random_state=42)

In [9]:
# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, name='coordinates')  # Output four coordinates: bonnet_tip_x, bonnet_tip_y, blowhead_x, blowhead_y
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
print(model.summary())

c:\Users\Aniket Konkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ coordinates (Dense)             │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,492 (56.61 MB)

 Trainable params: 14,839,492 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=8, batch_size=32)

Epoch 1/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 70s 541ms/step - loss: 238098.7031 - mae: 149.8178 - val_loss: 1544.4192 - val_mae: 31.2724
Epoch 2/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 69s 541ms/step - loss: 1582.8361 - mae: 31.1373 - val_loss: 1499.9969 - val_mae: 30.7850
Epoch 3/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 70s 544ms/step - loss: 1558.2332 - mae: 30.7228 - val_loss: 1491.2808 - val_mae: 29.9451
Epoch 4/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 68s 532ms/step - loss: 1510.3933 - mae: 30.4113 - val_loss: 1322.7518 - val_mae: 28.4265
Epoch 5/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 69s 539ms/step - loss: 1453.5109 - mae: 29.4832 - val_loss: 1299.6207 - val_mae: 28.1096
Epoch 6/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 69s 539ms/step - loss: 1323.3932 - mae: 28.0943 - val_loss: 1346.7640 - val_mae: 28.4298
Epoch 7/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 69s 538ms/step - loss: 1256.0134 - mae: 27.3509 - val_loss: 1420.6600 - val_mae: 29.8566
Epoch 8/8
128/128 ━━━━━━━━━━━━━━━━━━━━ 69s 538ms/step - loss: 1219.0929 - mae: 27.1555 - val_loss: 12

In [11]:
# Save model
save_model(model, "models/recognize_bonnet_blowhole#4.h5")

In [12]:
img_file_name_test, x_test = load_test_images_to_dataframe('dataset/imgs256x256#2')
x_test.shape

(6925, 256, 256, 3)

In [13]:
coordinates_pred = model.predict(x_test)

217/217 ━━━━━━━━━━━━━━━━━━━━ 26s 119ms/step


In [14]:
coordinates_pred

array([[124.15422 , 129.54317 , 124.98567 , 121.72155 ],
       [ 94.25281 ,  90.55403 ,  92.60582 ,  87.114296],
       [ 99.636154,  94.15767 ,  97.27659 ,  90.49315 ],
       ...,
       [124.4798  , 122.29283 , 123.801186, 119.22255 ],
       [ 98.16743 ,  92.3721  ,  95.66657 ,  88.691154],
       [110.08967 , 104.50507 , 106.63609 ,  97.04734 ]], dtype=float32)

### Save the Bonnet&Blowhead prediction to a CSV

In [15]:
bonnet_blowhead_coord_csv = pd.read_csv('dataset/sample_submission.csv')
bonnet_blowhead_coord_csv = bonnet_blowhead_coord_csv['Image']
coordinates_df = pd.DataFrame(coordinates_pred, columns=['bonnet_tip_x', 'bonnet_tip_y', 'blowhead_x', 'blowhead_y'])
bonnet_blowhead_coord_csv = pd.merge(bonnet_blowhead_coord_csv, coordinates_df, left_index=True, right_index=True)
bonnet_blowhead_coord_csv

,Image,bonnet_tip_x,bonnet_tip_y,blowhead_x,blowhead_y
0,w_1947.jpg,124.154221,129.543167,124.985672,121.721550
1,w_11096.jpg,94.252808,90.554031,92.605820,87.114296
2,w_10973.jpg,99.636154,94.157669,97.276588,90.493149
3,w_10442.jpg,134.705505,119.485237,128.255661,113.797165
4,w_10606.jpg,103.318024,112.961906,110.983177,122.898445
...,...,...,...,...,...
6920,w_4867.jpg,152.920807,142.218918,147.691254,134.214066
6921,w_5230.jpg,135.864639,122.683830,129.358459,114.147430
6922,w_5278.jpg,124.479797,122.292831,123.801186,119.222549
6923,w_9218.jpg,98.167427,92.372101,95.666573,88.691154


In [16]:
bonnet_blowhead_coord_csv.to_csv('annotations/predicted_bonnet_blowhead_coordinates#3.csv', index=False)